# Pointers

> A pointer references a location in memory, and obtaining the value stored at that location is known as
> dereferencing the pointer. As an analogy, a page number in a book's index could be considered a pointer to 
> the corresponding page; dereferencing such a pointer would be done by flipping to the page with the given 
> page number and reading the text found on that page.
>
> https://en.wikipedia.org/wiki/Pointer_(computer_programming)

A pointer is an object that stores the address of another object. If `T` is some type, then `T *` is a pointer that
can store the address an object of type `T`. Consider the following example:

```c
int *ptr;
```

`ptr` is a pointer object that can store the address of an object of type `int`. Most programmers would
simply say that `ptr` is a pointer to `int`.

There is no universally agreed on style for declaring pointer variables. Each of the following are legal
ways of declaring a pointer:

```c
int *ptr1;
int* ptr2;
int * ptr3;
```

None of the pointers `ptr1`, `ptr2`, or `ptr3` are initialized; thus, they almost certainly do not
point at valid `int` objects. Before using a pointer to perform some computation, the programmer must
ensure that the pointer actually points at a valid object.

## `NULL`

The name `NULL` defined in the header `<stddef.h>` means "no object". Any pointer variable can be assigned 
a value of `NULL` and such a pointer is called a null pointer:

```c
#include <stddef.h>

int *ptr = NULL;
```

The integer literal `0` can be used in place of `NULL`, but `NULL` is usually clearer in meaning.

It is important to note that there are no safe operations that can be performed with a null pointer
except to test if the pointer is `NULL` or not `NULL`. To test if a pointer `ptr` is a null pointer we
can use any of the following:

```c
if (!ptr) {
    // ptr is NULL
}

if (ptr == NULL) {
    // ptr is NULL
}

if (ptr == 0) {
    // ptr is NULL, but arguably less clear than using NULL
}
```

To test if a pointer `ptr` is not a null pointer we can use any of the following:

```c
if (ptr) {
    // ptr is not NULL
}

if (ptr != NULL) {
    // ptr is not NULL
}

if (ptr != 0) {
    // ptr is not NULL, but arguably less clear than using NULL
}
```

A null pointer can be used to indicate the absence of an object. In an array of pointer objects, a null
pointer can be used to indicate that there are no more elements in the array (similar to how `\0` is
used to indicate the end of a string). A null pointer may also be used to indicate an error condition.

## A pointer should refer to an object

A pointer stores the address of an object but how does the programmer obtain the address of an object?
The *address of* operator `&` creates a pointer that refers to an object:

```c
int a = 99;
int *ptr = &a;
```

In the example above, `&a` yields a pointer to the `int` stored in the variable `a`, and that pointer
is then assigned to the variable `ptr`.

`ptr` can store a pointer to any `int` object so it is perfectly legal to re-assign which object is pointed at:


```c
int a = 99;
int *ptr = &a;

int b = 55;
ptr = &b;
```


## Obtaining the value that the pointer refers to

The *pointer dereference* operator `*` dereferences a pointer to access the object that the pointer refers to.
The following example dereferences a pointer to compute the sum `a + 1`:

In [1]:
// ptr1.c

#include <stdio.h>

int main(void) {
    int a = 99;
    int *ptr = &a;        // ptr refers to a
    int b = *ptr + 1;     // b = a + 1
        
    printf("a = %d\n", a);
    printf("b = %d\n", b);
    
    return 0;
}

a = 99
b = 100


A value can be assigned to the object obtained via pointer dereferencing:

In [2]:
// ptr2.c 

#include <stdio.h>

int main(void) {
    int a = 99;
    int *ptr = &a;        // ptr refers to a
    *ptr = -33;           // assigns -33 to a
        
    printf("a = %d\n", a);
    
    return 0;
}

a = -33


Undefined behavior results if a null pointer is dereferenced:

In [4]:
// ptr3.c 

#include <stdio.h>

int main(void) {
    int *ptr = NULL;
    *ptr = -33;           // oops, null pointer dereference
        
    printf("a = %d\n", *ptr);
    
    return 0;
}

[C kernel] Executable exited with code -11

## `void` pointers

The pointer type `void *` can point to an object of any type:

In [9]:
int x = 1;
double y = 2;
char str[] = "hello";
    
void *p = &x;         // ok, p points to x
p = &y;               // ok, p points to y
p = str;              // ok, p points to first element of str

/tmp/tmpmygy9ivb.c: In function ‘main’:
/tmp/tmpmygy9ivb.c:8:7: warning: variable ‘p’ set but not used [-Wunused-but-set-variable]
    8 | void *p = &x;         // ok, p points to x
      |       ^


It is an error to attempt to dereference a void pointer because the compiler cannot determine the type
of the object being pointed at:

In [11]:
#include <stdio.h>

int main(void) {
    int x = 1;
    void *p = &x;
    
    int y = *p;      // error, dereference of void pointer
    
    return 0;
}

/tmp/tmpu7aqv87q.c: In function ‘main’:
/tmp/tmpu7aqv87q.c:7:13: warning: dereferencing ‘void *’ pointer
    7 |     int y = *p;
      |             ^~
/tmp/tmpu7aqv87q.c:7:13: error: void value not ignored as it ought to be
    7 |     int y = *p;
      |             ^
/tmp/tmpu7aqv87q.c:7:9: warning: unused variable ‘y’ [-Wunused-variable]
    7 |     int y = *p;
      |         ^
[C kernel] GCC exited with code 1, the executable will not be executed

The pointer can be dereferenced if it is cast to an appropriate type first:

In [12]:
#include <stdio.h>

int main(void) {
    int x = 1;
    void *p = &x;
    
    int y = *((int *) p);      // ok, dereferencing pointer to int
    printf("y = %d\n", y);
    
    return 0;
}

y = 1


One use of void pointers are for implementing generic data types.

# Using pointers to solve a common programming problem

A common programming problem is writing a function to change the value of one of its parameters. A simple
example is a function that swaps the values its two parameters. A first attempt at such a function might
look like so:

In [3]:
// badswap.c

#include <stdio.h>

void swap(double x, double y) {
    double tmp = x;
    x = y;
    y = tmp;
    printf("inside swap: x = %f, y = %f\n", x, y);
}

int main(void) {
    double a = 1.5;
    double b = 99.9;
    
    printf("before calling swap: a = %f, b = %f\n", a, b);
    swap(a, b);
    printf("after calling swap: a = %f, b = %f\n", a, b);
    
    return 0;
}

before calling swap: a = 1.500000, b = 99.900000
inside swap: x = 99.900000, y = 1.500000
after calling swap: a = 1.500000, b = 99.900000


Running the example above illustrates that the `swap` function manages to swap the values of the parameters
`x` and `y` but does not swap the values of the arguments `a` and `b`. The reason this is occurs is because
C uses pass-by-value to transfer the values of the arguments to the parameters of the function. In pass-by-value,
the value of the arguments are copied into the parameters and the function manipulates the parameters. The values
of the parameters are *not* copied back to the arguments at the end of the function.

For `swap` to swap the values of the arguments, `swap` would need to receive the actual objects corresponding
to the arguments. The solution in C is to pass a pointer to the objects that should be swapped:

In [4]:
// swap.c

#include <stdio.h>

void swap(double *x, double *y) {
    double tmp = *x;    // tmp gets the value of the object pointed to by x
    *x = *y;            // the object pointed to by x gets the value of the object pointed to by y
    *y = tmp;           // the object pointed to by y gets the value of tmp
    printf("inside swap: x = %f, y = %f\n", *x, *y);
}

int main(void) {
    double a = 1.5;
    double b = 99.9;
    
    printf("before calling swap: a = %f, b = %f\n", a, b);
    swap(&a, &b);      // calls swap with the address of a and the address of b
    printf("after calling swap: a = %f, b = %f\n", a, b);
}

before calling swap: a = 1.500000, b = 99.900000
inside swap: x = 99.900000, y = 1.500000
after calling swap: a = 99.900000, b = 1.500000


Another example of using a pointer as a function parameter is in a function that needs to return more than
one value. For example, it is often useful to return the maximum value and the index of the maximum value
when finding the maximum value in an array. One way to do so is to return one value using the return value
and the second value is set by dereferencing a pointer passed in by the caller:

In [3]:
// max.c

#include <stdio.h>

// Returns the maximum value in the array a having count elements.
// The index of the maximum element is stored in the object pointed at
// by index.
int max(int a[], size_t count, size_t *index) {
    int hi = a[0];
    size_t i_hi = 0;
    for (size_t i = 1; i < count; i++) {
        if (a[i] > hi) {
            hi = a[i];
            i_hi = i;
        }
    }
    *index = i_hi;
    return hi;
}

int main(void) {
    int arr[] = { 6, 3, 1, 8, 7, 9, 4, 2, 5 };
    
    // must declare a variable to store the index in
    size_t i;
    int big = max(arr, 9, &i);
    
    printf("max = %d, index = %lu\n", big, i);
    
    return 0;
}

max = 9, index = 5


A common mistake made by new C programmers when using a function such as `max` is to use a pointer that
does not point at an actual object. In the example below, the programmer has used a pointer object as
the argument to `max`, but the pointer does not actually point at a valid object:

In [4]:
// bad_call_to_max.c

#include <stdio.h>

// Returns the maximum value in the array a having count elements.
// The index of the maximum element is stored in the object pointed at
// by index.
int max(int a[], size_t count, size_t *index) {
    int hi = a[0];
    size_t i_hi = 0;
    for (size_t i = 1; i < count; i++) {
        if (a[i] > hi) {
            hi = a[i];
            i_hi = i;
        }
    }
    *index = i_hi;
    return hi;
}

int main(void) {
    int arr[] = { 6, 3, 1, 8, 7, 9, 4, 2, 5 };
    
    // oops, i is uninitialized and does not point at a valid object
    size_t *i;
    int big = max(arr, 9, i);
    
    printf("max = %d, index = %lu\n", big, i);
    
    return 0;
}

/tmp/tmp5calr9vw.c: In function ‘main’:
/tmp/tmp5calr9vw.c:26:15: warning: ‘i’ is used uninitialized [-Wuninitialized]
   26 |     int big = max(arr, 9, i);
      |               ^~~~~~~~~~~~~~
[C kernel] Executable exited with code -11

## Pointers and large objects

The `max` function in the previous section is an example of a function that has an array as one of its
parameters. We know that C uses pass-by-value when calling a function, so it would be reasonable to conclude
that calling `max` results in copying the contents of the caller's array into the array parameter `a` which
can be a time consuming operation if the array is large. Furthermore, making a copy of the array is wasteful
in this example because the array is never modified. Pointers solve the problem of passing and returning
large objects to and from functions. Instead of using pass-by-value, a pointer to the object can be passed
which means that a single pointer value must be copied instead of copying an entire object. When passing or
returning an array to or from a function, the array actually decays to a pointer to the first element
of the array (see the *Arrays* notebook for details).

## Pointer comparisons

Two pointers `p` and `q` are considered to be equal (`p == q` is true, `p != q` is false)
if any of the following is true:

* both pointers are null pointers of the same type
* both pointers point to the same object
* one pointer is a pointer to an array and the second pointer is a pointer to the first element of the
same array (see the *Arrays* notebook)
* one pointer is a pointer to a struct and the second pointer is a pointer to the first member of the
same struct (see the *Struct* notebook)
* both pointers are too-far pointers in the same array (see the *Arrays* notebook)

If none of the above are true, then the two pointers are considered to be unequal (`p == q` is false, 
`p != q` is true).

The following example illustrates the first two cases:

In [8]:

#include <stdio.h>

int main(void) {
    int x = 1;      // x and y have the same value but are different objects
    int y = 1;
    
    int *p = &x;    // p points at x
    int *q = &y;    // q points at y
    
    if (p == q) {
        puts("p equals q");
    }
    else {
        puts("p not equals q");
    }
    
    p = NULL;
    q = NULL;
    
    if (p == q) {
        puts("p equals q");
    }
    else {
        puts("p not equals q");
    }
    
    return 0;
}

p not equals q
p equals q


The operators `<`, `<=`, `>` and `>=` can also be used to compare pointers. See the *Arrays* notebook
for details.

## Pointer arithmetic

A pointer and integer can be summed or subtracted to produce a new pointer. Two pointers can be subtracted
to yield an integer value. These operations are useful when working with arrays. See the *Arrays* notebook
for details.